In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
import tqdm
import datetime
import os
import pickle

from glicko2 import Player

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [2]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [3]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [ ]:
players_df = dfs['players_df']
players_df.head()

In [ ]:
ranking_df = dfs['ranking_df']
ranking_df.head()

In [ ]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

In [ ]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data)")

sets_df.head()

In [ ]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

In [9]:
# Code optimization by Dan
# Basically we want to replace this line in process_tournament with something more efficient:
#
#      tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
#
# Instead, we can
# - Merge the tournament date info into ``sets_df``
# - Sort by date
# - Store the start/end positions of each tournament in a separate dictionary
# - Use tournament_sets_df = sets_df.iloc[start:end+1] instead.

sets_df = sets_df.merge(tournament_info_df[['key', 'start', 'end']], left_on='tournament_key', right_on='key', how='left')
sets_df = sets_df.drop(labels=['key_y'], axis='columns')
sets_df = sets_df.rename(columns={"key_x": "key"})
sets_df = sets_df.sort_values(by=['end', 'tournament_key']) # Just in case there are tournaments with the exact same end date

In [ ]:
sets_df.head(1)

In [103]:
game_data_sets_df = sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)]

In [ ]:
# Here, for each row in game_data_sets_df,
# we will replace it with multiple rows,
# one for each entry in 'game_data',
# with other row entries updated accordingly

# No point in keeping track of the game data after this
new_columns = list(game_data_sets_df.columns)
new_columns.remove('game_data')

# Primitive progress bar so we don't get bored waiting
counter = 0

# One row -> multiple rows as above
def duplicate_row(x):
    global counter

    # Create a new row for each entry in ['game_data']
    # Store the result here
    games = []

    for game_info in x.iloc[0]['game_data']:
        game = x.iloc[0][new_columns].copy()

        # Invalid character data, just skip the row 
        if game_info['winner_char'] is None or game_info['loser_char'] is None:
            continue

        # This helps deal with invalid player data in ['game_data']
        # on the off chance that it exists
        winner_id = game['p1_id'] if game['winner_id'] == game['p1_id'] else game['p2_id']
        loser_id  = game['p2_id'] if game['winner_id'] == game['p1_id'] else game['p1_id']

        # Update the player ids to include the character that they played
        winner_pc = winner_id + '/' + game_info['winner_char']
        loser_pc  = loser_id  + '/' + game_info['loser_char']

        # Change ids and scores in this row to reflect that specific game
        if game['winner_id'] == game['p1_id']:
            game['p1_id'] = winner_pc
            game['p1_score'] = game_info['winner_score']
            game['p2_id'] = loser_pc
            game['p2_score'] = game_info['loser_score']
        else:
            game['p1_id'] = loser_pc
            game['p1_score'] = game_info['loser_score']
            game['p2_id'] = winner_pc
            game['p2_score'] = game_info['winner_score']

        game['winner_id'] = winner_pc

        games.append(game)

    # Progress bar
    if counter % 10000 == 0:
        print(counter)

    counter += 1

    return pd.DataFrame(games, columns=x.columns)

# More janky nonsense.
# reset_index() to get an index to groupby().
# Then groupby() on that index (yes, one row per group),
#     BECAUSE .apply() on groupby can take a function that returns a dataframe with a different number of rows
processed_game_data_sets = game_data_sets_df.reset_index().groupby('index').apply(duplicate_row, include_groups=False).reset_index(drop=True)
processed_game_data_sets[(processed_game_data_sets['p1_id'] == '1021/melee/yoshi') | (processed_game_data_sets['p2_id'] == '1021/melee/yoshi')]

In [ ]:
# pd.to_pickle(processed_game_data_sets, data_path + 'individual_game_data.pkl')
individual_game_data_df = pd.read_pickle(data_path + 'individual_game_data.pkl')
individual_game_data_df = individual_game_data_df.sort_values('end')
individual_game_data_df

In [12]:
# Because it would take an ABSURDLY long time to compute the ELOs of
# (specific_player)/(specific_character) \union (anyone)/(specific_opponent_character)
# we will instead approximate as follows:
# 1. compute the ELOs of (anyone)/(specific_opponent_character) (ex: fox vs fox, to guage vaguely how good each fox player is)
# 2. Having those values fixed, compute the ELO of (specific_player)/(specific_character) (ex: aMSa/yoshi vs all fixed fox player ELOs)
#    in their matches against any such player from before.

individual_game_data_df['winner_char'] = individual_game_data_df['winner_id'].apply(lambda x: x.split('/')[-1])
individual_game_data_df['p1_char'] = individual_game_data_df['p1_id'].apply(lambda x: x.split('/')[-1])
individual_game_data_df['p2_char'] = individual_game_data_df['p2_id'].apply(lambda x: x.split('/')[-1])

all_characters = list(set(list(individual_game_data_df['p1_char'].unique()) + list(individual_game_data_df['p1_char'].unique())))

In [ ]:
char_vs_char_dataframes = {}

for character in all_characters:
    char_vs_char_df = individual_game_data_df[(individual_game_data_df['p1_char'] == character) & (individual_game_data_df['p2_char'] == character)]
    char_vs_char_df = char_vs_char_df.reset_index(drop=True) # Necessary for proper position calculations later
    
    char_vs_char_dataframes[character] = char_vs_char_df

char_vs_char_dataframes['fox'].head()

In [ ]:
# Dictionary of dictionaries. First key is the specific character used for the matchup.
# For each one, we will store the start/end indexes of each tournament in each dataframe.
# This will be used for far more efficient processing later
tournament_game_positions_by_char = {}

for character in all_characters:
    positions = {}
    char_vs_char_df = char_vs_char_dataframes[character]

    # Once more, just to be absolutely sure things are formatted properly
    char_vs_char_df = char_vs_char_df.sort_values(by=['end', 'tournament_key'])

    # This is gonna take a while. Might as well show a progress bar
    for i in tqdm.tqdm(range(0, len(char_vs_char_df.index)), total=len(char_vs_char_df.index)):
        key = char_vs_char_df.iloc[i]['tournament_key']
        if key not in positions:
            positions[key] = (i,i)
        else:
            positions[key] = (positions[key][0], i)

    tournament_game_positions_by_char[character] = positions

In [ ]:
char_vs_char_dataframes['fox'].head()

In [ ]:
# TLDR the ELO computation occasionally seems to go negative, and I think this can cause a lot of problems.
# Likewise, the RD value seems to get unusually large occasionally
MIN_ELO = 500.0
MAX_RD = 350.0

# Basically, we will instead create a dataframe with index of dates, columns of players
character = 'fox'
games_df = char_vs_char_dataframes[character]
positions = tournament_game_positions_by_char[character]

unique_players = list(set(list(games_df['p1_id'].unique()) + list(games_df['p2_id'].unique())))

# Let's get the dates of each period (tournament) and, importantly, know how many there are
# This will be useful for pre-allocating the entire pandas dataframe rather than constantly building it row by row
# NOTE: Some tournaments might have the same end date. That's ok.
#       The algorithm will simply update those entries multiple times with newer and newer glicko values.
tournament_list = [x for x in positions]
dates = list(set([games_df.iloc[positions[tournament][0]]['end'] for tournament in tournament_list])) # VERY much need these to be unique
dates.sort() # Just to be absolutely 100% sure, but it probably stays sorted with the above operation anyways

# Convenient store of glicko objects
glicko_objects = {}
for player in unique_players:
    glicko_objects[player] = Player()

# Initial row of default ELO values, at the earliest possible date.
# Note that we are also pre-allocating the dataframe with the rest of the possible dates.
player_ratings_df = pd.DataFrame([[1500.0] * len(unique_players)], columns=unique_players, index=[datetime.datetime(2015, 1, 11, 14, 16, 0)] + dates)

def process_tournament(player_ratings_df, games_df, positions, tournament_key, glicko_objects):
    # Get the games for this specific period, using precomputed indices for speed.
    period_df = games_df.iloc[positions[tournament_key][0]:positions[tournament_key][1]+1]

    # Prepare player matches DataFrame.
    # Essentially, one copy where p1 is the player of consideration,
    # and then another copy where p2 is under consideration.
    df_p1 = period_df[['p1_id', 'p2_id', 'winner_id']].copy()
    df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id'}, inplace=True)
    df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
    df_p1 = df_p1[['player_id', 'opponent_id', 'outcome']]
    
    df_p2 = period_df[['p2_id', 'p1_id', 'winner_id']].copy()
    df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id'}, inplace=True)
    df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
    df_p2 = df_p2[['player_id', 'opponent_id', 'outcome']]

    outcomes_df = pd.concat([df_p1, df_p2], ignore_index=True)

    # Add opponent ELO and RD values
    outcomes_df['opponent_rating'] = outcomes_df['opponent_id'].apply(lambda x: glicko_objects[x].getRating())
    outcomes_df['opponent_rd'] = outcomes_df['opponent_id'].apply(lambda x: glicko_objects[x].getRd())

    # Compile the opponent data into lists (one for ratings, one for rds, one for outcomes) for every player
    grouped = outcomes_df.groupby('player_id').agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()

    # We will need to update the glicko objects separately for players that did or didn't play in this rating period
    players_with_games = list(grouped['player_id'])
    players_without_games = [x for x in unique_players if x not in players_with_games]

    #First, let's handle the players that did play games in this period
    def update_glicko(row):
        glicko_objects[row['player_id']].update_player(row['opponent_rating'], row['opponent_rd'], row['outcome'])

    grouped.apply(update_glicko, axis=1)

    # Now let's handle players that did NOT play games in this period
    for player in players_without_games:
        glicko_objects[player].did_not_compete()

    # This might be a bug in the glicko2 library that we are using,
    # but occasionally we get weird values for the rating and/or RD
    # and this DOES very occasionally seem to cause the whole thing to crash
    # (division by zero somewhere)
    # and so we use professional-grade duct tape here
    for player in unique_players:
        if glicko_objects[player].getRating() < MIN_ELO:
            glicko_objects[player].setRating(MIN_ELO)

        if glicko_objects[player].getRd() > MAX_RD:
            glicko_objects[player].setRd(MAX_RD)

    # Now update that specific row of the player ratings dataframe with the new glicko data
    date = games_df.iloc[positions[tournament_key][0]]['end']

    #new_player_data = {}
    #for player in unique_players:
    #    new_player_data[player] = glicko_objects[player].getRating()

    # These should be in the same order as the columns, as unique_players was used to create the columns as well.
    # This is slightly more efficient than using a dictionary.
    new_player_data = [glicko_objects[player].getRating() for player in unique_players]

    player_ratings_df.loc[date] = new_player_data

for tournament_key in tqdm.tqdm(tournament_list, total=len(tournament_list)):
    process_tournament(player_ratings_df, games_df, positions, tournament_key, glicko_objects)

player_ratings_df

In [ ]:
#games_df.head()
list(player_ratings_df['4165/melee/fox'])

In [ ]:
# TODO: Our processed_game_data_sets now contains end dates directly. Remove the tournament_info_df in the arguments
def process_tournament(tournament_key, player_ratings_df, tournament_info_df, processed_game_data_sets, positions):
    """
    Process a tournament to update player ratings.
    """
    # Get the sets for this tournament
    tournament_sets_df = processed_game_data_sets.iloc[positions[tournament_key][0]:positions[tournament_key][1]+1]
    
    # Extract the unique player IDs from the sets
    tournament_players = pd.unique(tournament_sets_df[['p1_id', 'p2_id']].values.ravel())
   

    # Add any new players to the player_ratings_df
    new_players = [player for player in tournament_players if player not in player_ratings_df.index]
    if new_players:
        new_player_df = pd.DataFrame({
            'dates': [[datetime.datetime(2015, 1, 11, 14, 16, 0)] for _ in new_players],
            'rating_history': [[1500.0] for _ in new_players],
            'rd_history': [[350.0] for _ in new_players],
            'glicko2': [Player() for _ in new_players]
        }, index=new_players)
        # Check the IDs of the Player objects
    
        player_ratings_df = pd.concat([player_ratings_df, new_player_df], ignore_index=False)
    

    # Ensure the index name is set to 'player_id'
    player_ratings_df.index.name = 'player_id'

    # Create a mapping from player_id to their Glicko2 player object
    player_map = player_ratings_df.loc[tournament_players]['glicko2'].to_dict()

    # Create a snapshot of ratings before the tournament
    ratings_snapshot = player_ratings_df.loc[tournament_players][['glicko2']].copy()
    ratings_snapshot['rating'] = ratings_snapshot['glicko2'].apply(lambda x: x.getRating())
    ratings_snapshot['rd'] = ratings_snapshot['glicko2'].apply(lambda x: x.getRd())
    # Add 'opponent_id' column from the index
    ratings_snapshot.reset_index(inplace=True)
    ratings_snapshot.rename(columns={'player_id': 'opponent_id'}, inplace=True)
    ratings_snapshot = ratings_snapshot[['opponent_id', 'rating', 'rd']]
    # print(ratings_snapshot)
    

    # Prepare player matches DataFrame
    df_p1 = tournament_sets_df[['p1_id', 'p2_id', 'winner_id']].copy()
    df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id'}, inplace=True)
    df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
    df_p1 = df_p1[['player_id', 'opponent_id', 'outcome']]
    
    df_p2 = tournament_sets_df[['p2_id', 'p1_id', 'winner_id']].copy()
    df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id'}, inplace=True)
    df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
    df_p2 = df_p2[['player_id', 'opponent_id', 'outcome']]
    
    player_matches = pd.concat([df_p1, df_p2], ignore_index=True)
    
    # Merge to get opponent ratings and RDs
    player_matches = player_matches.merge(
        ratings_snapshot,
        on='opponent_id', how='left'
    )
    player_matches.rename(columns={'rating': 'opponent_rating', 'rd': 'opponent_rd'}, inplace=True)
    
    # Group by player_id to aggregate opponent data
    grouped = player_matches.groupby('player_id').agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()
    
    # Get the tournament end date
    end_date = tournament_info_df.loc[tournament_info_df['key'] == tournament_key, 'end'].values[0]
    end_date = pd.to_datetime(end_date)
    
    def process_player(row):
            player_id = row['player_id']
            rating_list = row['opponent_rating']
            rd_list = row['opponent_rd']
            outcome_list = row['outcome']
            
            # Update Glicko rating for the player
            player_glicko = player_map[player_id]
            if rating_list:  # Ensure the player has matches to process
                player_glicko.update_player(rating_list, rd_list, outcome_list)
            
            # Update the player's history
            player_ratings_df.at[player_id, 'dates'].append(end_date)
            player_ratings_df.at[player_id, 'rating_history'].append(player_glicko.getRating())
            player_ratings_df.at[player_id, 'rd_history'].append(player_glicko.getRd())
            
        # Apply the function to each row using apply
    grouped.apply(process_player, axis=1)
    
    return player_ratings_df  # Return the updated DataFrame

#sorted_tournament_info_df = tournament_info_df.sort_values('end').reset_index(drop=True)
#sorted_tournament_info_df.shape[0]

ratings_by_char = {}

for character in all_characters:
    # Initialize player ratings DataFrame
    player_ratings_df = pd.DataFrame(columns=[
        'dates', 'rating_history', 'rd_history', 'glicko2'
    ])

    char_vs_char_df = char_vs_char_dataframes[character]
    positions = tournament_game_positions_by_char[character]

    # Loop over tournaments
    # NOTE: Now we have to only consider tournaments that actually have sets
    print(character)
    tournament_list = [x for x in positions]
    for tournament_key in tqdm.tqdm(tournament_list, total=len(tournament_list)):
        player_ratings_df = process_tournament(tournament_key, player_ratings_df, tournament_info_df, char_vs_char_df, positions)

    ratings_by_char[character] = player_ratings_df  


In [ ]:
ratings_by_char['fox']

In [ ]:
plt.plot(player_ratings_df.loc['1021/melee/yoshi']['dates'], player_ratings_df.loc['1021/melee/yoshi']['rating_history'], label='aMSa')
plt.plot(player_ratings_df.loc['19554/melee/fox']['dates'], player_ratings_df.loc['19554/melee/fox']['rating_history'], label='Cody')
plt.legend()

In [ ]:
# Load the DataFrame to verify
loaded_df = pd.read_pickle(data_path + 'overall_players_ranking.pkl')
loaded_df.head()

In [ ]:
plt.plot(loaded_df.loc['1021']['dates'], loaded_df.loc['1021']['rating_history'], label='aMSa')
plt.plot(loaded_df.loc['19554']['dates'], loaded_df.loc['19554']['rating_history'], label='Cody')
plt.legend()

In [ ]:
max_rating = 0

for index, row in loaded_df.iterrows():
    index_max_rating = np.argmax(row['rating_history'].astype(int))  # Correcting the typo to `astype`
    if row['rating_history'][index_max_rating] > max_rating:
        best_row = row
        best_index = index
        max_rating = row['rating_history'][index_max_rating]
        print(players_df[players_df['player_id'] == index]['tag'], max_rating)
        # print(max_rating)

print(max_rating)


In [ ]:
players_df[players_df['tag'] == 'Cody Schwab']